![CoSAppLogo](../images/cosapp.svg)

**CoSApp** tutorials on optimization

# Sellar optimization problem

Multi Domain Optimization (MDO) problem proposed by Sellar et al. in

Sellar, R., Batill, S., & Renaud, J. (1996).
*Response surface based, concurrent subspace optimization for multidisciplinary system design.*
In 34th Aerospace Sciences Meeting and Exhibit (p. 714).

The MDO problem is written as follows:

Minimize $x^2 + z_{2} + y_1^2 + e^{-y_2}$ with respect to $x,\,z$, where:

- $y_1 = z_{1}^2 + z_{2} + x - 0.2\,y_2$
- $y_2 = \sqrt{y_1} + z_{1} + z_{2}$

subject to general constraints:

- $0 \leq z_{1} \leq 10$
- $0 \leq z_{2} \leq 10$
- $0 \leq x \leq 10$
- $y_1 \geq 3.16$
- $y_2 \leq 24$


In [ ]:
from cosapp.base import System
import numpy as np


class SellarDiscipline1(System):
    """Component containing Discipline 1.
    """
    def setup(self):
        self.add_inward('z', np.zeros(2))
        self.add_inward('x')
        self.add_inward('y2')

        self.add_outward('y1')

    def compute(self):
        """Evaluates equation
        y1 = z1**2 + z2 + x - 0.2 * y2
        """
        self.y1 = self.z[0]**2 + self.z[1] + self.x - 0.2 * self.y2


class SellarDiscipline2(System):
    """Component containing Discipline 2.
    """
    def setup(self):
        self.add_inward('z', np.zeros(2))
        self.add_inward('y1')

        self.add_outward('y2')

    def compute(self):
        """Evaluates equation
        y2 = sqrt(|y1|) + z1 + z2
        """
        self.y2 = np.sqrt(abs(self.y1)) + self.z[0] + self.z[1]


In [ ]:
class Sellar(System):
    """System modeling the Sellar case.
    """
    def setup(self):
        d1 = self.add_child(SellarDiscipline1('d1'), pulling=['x', 'z', 'y1'])
        d2 = self.add_child(SellarDiscipline2('d2'), pulling=['z', 'y2'])
        
        # Couple sub-systems d1 and d2:
        self.connect(d1, d2, ['y1', 'y2'])


## Define optimization problem

In [ ]:
import time
from cosapp.drivers import Optimizer, NonLinearSolver

s = Sellar('s')

optim = s.add_driver(Optimizer('optim', method='SLSQP', tol=1e-12, verbose=1))
optim.add_child(NonLinearSolver('solver', tol=1e-12))  # to solve cyclic dependencies

# Set optimization problem
optim.set_minimum('x**2 + z[1] + y1 + exp(-y2)')
optim.add_unknown(['x', 'z'])
optim.add_constraints([
    'y1 >= 3.16',
    'y2 <= 24',
    '0 <= x <= 10',
    '0 <= z <= 10',
])

optim

## Solve problem

In cell below, we initialize the system, and solve our optimization problem.
Beforehand, we add a recorder and turn on option `monitor`, to keep a record of the system state at all iterations.

In [ ]:
from cosapp.recorders import DataFrameRecorder

# Add recorder to monitor iterations
optim.add_recorder(
    DataFrameRecorder(
        includes = ["*", "drivers['optim'].objective"],
        excludes = "d?.*",
    )
)
optim.options['monitor'] = True

# Initialization
s.x = 1.0
s.z = np.array([5.0, 2.0])
s.y1 = 1.0
s.y2 = 1.0
s.run_once()

# Run simulation
# s.exec_order = ('d2', 'd1')  # should not change results
start_time = time.time()
s.run_drivers()

print(f"Time: {time.time() - start_time:.3f} s")

for varname in ('x', 'z', 'y1', 'y2'):
    print(f"s.{varname} = {s[varname]}")

print(f"objective = {optim.objective}")
print(f"s.y1 = 3.16 + {s.y1 - 3.16}")

## Analyze convergence

In [ ]:
df = optim.recorder.export_data()
df = df.rename(columns={"drivers['optim'].objective": "objective"})
df.drop(['Section', 'Status', 'Error code', 'Reference'], axis=1)

In [ ]:
import plotly.express as px

fig = px.line(df, y="objective",
    hover_data = {
        'x': True,
        'z': True,
        'y1': True,
        'y2': True,
    },
    title = "Convergence",
    labels = {'index': 'iteration'}
)
fig.update_layout(
    height = 600,
    hovermode = 'x',
)
fig.update_traces(
    mode = 'lines+markers',
)
fig.show()

## Alternative formulation

Specify constraints on unknowns with arguments `lower_bounds` and `upper_bounds` in `Optimizer.add_unknown`.
This alternative declaration may lead to a faster and more stable resolution, as bounds are not treated as nonlinear constraints.

In [ ]:
import time
from cosapp.drivers import Optimizer, NonLinearSolver

s = Sellar('s')

optim = s.add_driver(Optimizer('optim', method='SLSQP', tol=1e-12, verbose=True))
optim.add_child(NonLinearSolver('solver', tol=1e-12))  # to solve cyclic dependencies

# Set optimization problem
optim.set_minimum('x**2 + z[1] + y1 + exp(-y2)')
optim.add_unknown('x', lower_bound=0, upper_bound=10)
optim.add_unknown('z', lower_bound=0, upper_bound=10)
optim.add_constraints([
    'y1 >= 3.16',
    'y2 <= 24',
])

# Initialization
s.x = 1.0
s.z = np.array([5.0, 2.0])
s.y1 = 1.0
s.y2 = 1.0
s.run_once()

# Run simulation
# s.exec_order = ('d2', 'd1')  # should not change results
start_time = time.time()
s.run_drivers()

print(f"Time: {time.time() - start_time:.3f} s")

for varname in ('x', 'z', 'y1', 'y2'):
    print(f"s.{varname} = {s[varname]}")

print(f"objective = {optim.objective}")
print(f"y1 = 3.16 + {s.y1 - 3.16}")